In [156]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [370]:
soc = pd.read_csv("Data/Sociodemographic Data/CMAP/ReferenceCCAProfiles20152019.csv")
covid = pd.read_csv("Data/Covid Data/Medical_Examiner_Case_Archive_-_COVID-19_Related_Deaths.csv")
zip = pd.read_csv("Data/Zip_Codes.csv")

In [371]:
zipcodes = zip.ZIP.to_list()
covid = covid[covid['Residence Zip'].notna()]
sep = '-'
covid['Residence Zip'] = covid['Residence Zip'].apply(lambda x: x.split(sep, 1)[0])
covid['Residence Zip'] = covid['Residence Zip'].astype(str).astype(int)
covid = covid[covid['Residence Zip'].isin(zipcodes)]

In [372]:
covid

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Residence City,Residence Zip,Manner of Death,...,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,OBJECTID,Chicago Ward,Chicago Community Area
1,ME2020-02532,03/25/2020 09:00:00 AM,04/11/2020 05:45:00 PM,75.0,Female,Asian,False,Chicago,60616,NATURAL,...,NaN,2732 North Hampton Court,CHICAGO,60614,NaN,NaN,NaN,35295,NaN,NaN
2,ME2020-02887,04/11/2020 12:00:00 AM,04/15/2020 08:26:00 PM,80.0,Male,Black,False,Chicago,60640,NATURAL,...,NaN,4920 N. Kenmore/ Uptown Health Ctr.,CHICAGO,60640,NaN,NaN,NaN,35680,NaN,NaN
5,ME2020-04155,03/29/2020 06:30:00 PM,04/29/2020 10:20:00 PM,90.0,Male,Black,False,Chicago,60619,NATURAL,...,NaN,8108 s Eberhardt Ave,CHICAGO,60619,NaN,NaN,NaN,36941,NaN,NaN
6,ME2020-04573,04/15/2020 11:56:00 AM,05/03/2020 03:40:00 PM,79.0,Male,White,False,Chicago,60626,NATURAL,...,NaN,7445 n. Sherican Road,CHICAGO,60626,NaN,NaN,NaN,37325,NaN,NaN
8,ME2020-04807,04/30/2020 12:00:00 PM,05/07/2020 12:34:00 AM,90.0,Male,White,True,Chicago,60646,NaN,...,NaN,"Peterson Park NH, 6141 N. Pulaski Road",CHICAGO,60646,NaN,NaN,NaN,37567,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15163,ME2021-11778,12/16/2021 10:22:00 AM,12/16/2021 10:22:00 AM,89.0,Female,Black,False,Chicago,60620,NATURAL,...,3.0,7725 SOUTH SEELY,CHICAGO,60620,-87.674335,41.753398,"(41.75339764, -87.67433543)",60665,18.0,AUBURN GRESHAM
15165,ME2021-12512,12/30/2021 10:31:00 AM,12/30/2021 11:47:00 AM,80.0,Male,White,True,Chicago,60608,NATURAL,...,7.0,2610 W 25TH ST,CHICAGO,60608,-87.690945,41.846695,"(41.84669495, -87.69094472)",61389,12.0,SOUTH LAWNDALE
15166,ME2021-12218,12/13/2021 04:30:00 PM,12/25/2021 03:17:00 AM,71.0,Female,White,True,Chicago,60623,NATURAL,...,7.0,2848 S. CENTRAL PARK AVE.,CHICAGO,60623,-87.714903,41.839273,"(41.83927325, -87.71490343)",61092,22.0,SOUTH LAWNDALE
15167,ME2021-09220,09/23/2021 01:45:00 PM,10/02/2021 07:15:00 AM,86.0,Female,White,False,Chicago,60632,NATURAL,...,7.0,5420 South Sawyer Ave,CHICAGO,60632,-87.705078,41.794679,"(41.79467922, -87.70507754)",58103,14.0,GAGE PARK


In [274]:
# 3% credit
source = requests.get('https://censusreporter.org/profiles/86000US60612-60607/').text

soup = BeautifulSoup(source, 'lxml')

s = soup.findAll('script',type="text/javascript")[1].getText()

In [294]:
def parse(param,sp):
    param = '"'+param+'", '
    try:
        var = float(re.findall(param+'"values": {"this":\s*([+-]?[0-9]+\.[0-9]+)',sp)[0])
    except:
        var = 0
    
    return var

In [328]:
median_age = parse("Median age",s)

percent_male = parse("Male",s)
percent_female = parse("Female",s)

percent_white = parse("White",s)
percent_black = parse("Black",s)
percent_native = parse("Native",s)
percent_asian = parse("Asian",s)
percent_islander = parse("Islander",s)
pecent_two_plus = parse("Two\+",s)
percent_hispanic = parse("Hispanic",s)

per_capita = parse("Per capita income", s)
median_household_income = parse("Median household income",s)

percent_below_poverty = parse("Persons below poverty line",s)

mean_travel_time = parse("Mean travel time to work",s)
drove_alone = parse("Drove alone",s)
carpooled = parse("Carpooled",s)
public_transit = parse("Public transit",s)
bicycle = parse("Bicycle",s)
walked = parse("Walked",s)
other = parse("Other",s)
worked_at_home = parse("Worked at home",s)

number_of_households = parse("Number of households",s)
persons_per_household = parse("Persons per household",s)

married = parse("Married",s)
single = parse("Single",s)

number_of_housing_units = parse("Number of housing units",s)
occupied_housing_units = parse("Occupied",s)
vacant_housing_units = parse("Vacant",s)
owner_housing_units = parse("Owner occupied",s)
renter_housing_units = parse("Renter occupied",s)

median_value_owner_occupied = parse("Median value of owner-occupied housing units",s)

moved_since_previous_year = parse("Moved since previous year",s)
same_house_year_ago = parse("Same house year ago",s)

no_degree = parse("No degree",s)
high_school = parse("High school",s)
some_college = parse("Some college",s)
bachelors = parse("Bachelor's",s)
post_grad = parse("Post-grad",s)

foriegn_born_pop = parse("Foreign-born population",s)
europe = parse('Europe',s)
asia = parse('Asia',s)
africa = parse('Africa',s)
oceania = parse('Oceania',s)
latin_america = parse('Latin America',s)
north_america = parse('North America',s)

"Islander", 
